# Introducción

Descripción del proyecto
La compañía minera OilyGiant requiere encontrar el mejor lugar para un nuevo pozo.
Para elegir la ubicación se llevan a cabo los siguientes pasos:
- Se recolectan los parámetros del pozo de petróleo en la región seleccionada: calidad del petróleo y volumen de reservas
- Se construye un modelo para predecir el volumen de reservas en los nuevos pozos
- Se seleccionan los pozos de petróleo con los valores estimados más altos;
- Se elige la región con el mayor beneficio total para los pozos de petróleo seleccionados.

Se cuenta con datos sobre muestras de crudo de tres regiones. Ya se conocen los parámetros de cada pozo petrolero de la región. Se creará un modelo que ayude a elegir la región con el mayor margen de beneficio. Se analizarán los beneficios y riesgos potenciales utilizando la técnica bootstrapping


# Tabla de contenidos

* [1- Importación de librerias y carga de datos](#chapter1)

    
* [2 - Entrenamiento y prueba del modelo para cada región](#chapter2)

 
* [3- Cálculo de ganancias](#chapter3)

  
* [4- Calculo de ganancia de pozos de petróleo seleccionados](#chapter4)


* [5-  Calculo de riesgos y ganancias para cada región](#chapter5)


* [6- Conclusiones](#chapter6)



## Importación de librerias y carga de datos <a class="anchor" id="chapter1"></a>


In [46]:
# importacion de librerias

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

In [47]:
#CArga de datos
geo_data_0 = pd.read_csv('/datasets/geo_data_0.csv')
geo_data_1 = pd.read_csv('/datasets/geo_data_1.csv')
geo_data_2 = pd.read_csv('/datasets/geo_data_2.csv')


In [48]:
#Observacion general de datos
print(geo_data_0.head(2))
print(geo_data_1.head(2))
print(geo_data_2.head(2))

      id        f0        f1       f2     product
0  txEyH  0.705745 -0.497823  1.22117  105.280062
1  2acmU  1.334711 -0.340164  4.36508   73.037750
      id         f0        f1        f2    product
0  kBEdx -15.001348 -8.276000 -0.005876   3.179103
1  62mP7  14.272088 -3.475083  0.999183  26.953261
      id        f0        f1        f2    product
0  fwXo0 -1.146987  0.963328 -0.828965  27.758673
1  WJtFt  0.262778  0.269839 -2.530187  56.069697


In [49]:
print(geo_data_0.info())
print(geo_data_1.info())
print(geo_data_2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column  

- No hay valores ausentes en los datos y son del tipo correcto

In [50]:
# Se separan los datos en caracteristicas y objetivos ( se excluyen los datos de ID)

features_0 = geo_data_0.drop(['id', 'product'], axis=1)
target_0= geo_data_0['product']

features_1 = geo_data_1.drop(['id', 'product'], axis=1)
target_1 = geo_data_1['product']

features_2 = geo_data_2.drop(['id', 'product'], axis=1)
target_2 = geo_data_2['product']

In [51]:
# Se dividen los datos en conjunto de entrenamiento y validacion 

In [52]:
features_train_0, features_valid_0, target_train_0, target_valid_0 = train_test_split(features_0, target_0, test_size = 0.25, random_state= 12345)

features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(features_1, target_1, test_size = 0.25, random_state= 12345)

features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(features_2, target_2, test_size = 0.25, random_state= 12345)




In [53]:
# Se escalan las caracteristicas numericas (por ser que se utilizara un modelo basado en regresion)

scaler_0 = StandardScaler()
scaler_0.fit(features_train_0)
features_train_0 = scaler_0.transform(features_train_0)
features_valid_0 = scaler_0.transform(features_valid_0)

scaler_1 = StandardScaler()
scaler_1.fit(features_train_1)
features_train_1 = scaler_1.transform(features_train_1)
features_valid_1 = scaler_1.transform(features_valid_1)

scaler_2 = StandardScaler()
scaler_2.fit(features_train_2)
features_train_2 = scaler_2.transform(features_train_2)
features_valid_2 = scaler_2.transform(features_valid_2)


## Entrenamiento y prueba del modelo para cada región <a class="anchor" id="chapter2"></a>

In [54]:
#Funcion para el entrenamiento y cálculo de predicciones con modelo de regresion lineal

def predictions (features_train, target_train, features_valid):
    model = LinearRegression()# inicializacion del constructor de modelo
    model.fit(features_train, target_train) # entrenamiento del modelo en el conjunto de entrenamiento
    predictions_valid = model.predict(features_valid) #  predicciones del modelo en el conjunto de validación
    return predictions_valid

#Aplicacion de la funcion a las 3 regiones y calculo del RECM

predictions_valid_0 = predictions(features_train_0, target_train_0, features_valid_0)
result_0 = mean_squared_error(target_valid_0, predictions_valid_0)**0.5
print( "RECM del modelo de region_0 en el conjunto de validación:", result_0)

predictions_valid_1 = predictions(features_train_1, target_train_1, features_valid_1)
result_1 = mean_squared_error(target_valid_1, predictions_valid_1)**0.5
print( "RECM del modelo de region_1 en el conjunto de validación:", result_1)

predictions_valid_2 = predictions(features_train_2, target_train_2, features_valid_2)
result_2 = mean_squared_error(target_valid_2, predictions_valid_1)**0.5
print( "RECM del modelo de region_2 en el conjunto de validación:", result_2)

RECM del modelo de region_0 en el conjunto de validación: 37.5794217150813
RECM del modelo de region_1 en el conjunto de validación: 0.893099286775617
RECM del modelo de region_2 en el conjunto de validación: 69.2732377722098


 - El modelo de la region 1 cuenta con el menor valor de RECM

## Cálculo de ganancias <a class="anchor" id="chapter3"></a>

In [55]:
#Calculo del volumen de reservas suficiente para desarrollar un nuevo pozo sin pérdidas

inversion_por_pozo = 100000000/200
ingreso_unidad_reserva = 4500
reservas_minima_req_pozo = inversion_por_pozo/ingreso_unidad_reserva
print("Reservas mínimas requeridas para desarrollar pozo sin perdida:" , round(reservas_minima_req_pozo))


Reservas mínimas requeridas para desarrollar pozo sin perdida: 111


In [56]:
#Calculo de media de reservas por pozo por region

print('Volumen medio de reservas por pozo en Region 0:', round(predictions_valid_0.mean()))
print('Volumen medio de reservas por pozo en Region 1:', round(predictions_valid_1.mean()))
print('Volumen medio de reservas por pozo en Region 2:', round(predictions_valid_2.mean()))

Volumen medio de reservas por pozo en Region 0: 93
Volumen medio de reservas por pozo en Region 1: 69
Volumen medio de reservas por pozo en Region 2: 95


- El volumen medio de reservas por pozo estimado por el modelo (regresion lineal) para cada region no supera el minimo requerido para desarrollar un pozo sin perdida

## Calculo de ganancia de pozos de petróleo seleccionados <a class="anchor" id="chapter4"></a>

In [57]:
#FUncion para el calculo de ingresos estimados para los pozos de mayor reservas

def revenues (predicted_valid, target_valid, count, unit_income, cost_pozo):
    predicted_valid_sorted = predicted_valid.sort_values(ascending = False)
    total_reservas_seleccion = target_valid.reset_index(drop=True).iloc[predicted_valid_sorted.index][:count].sum()
    income_total = total_reservas_seleccion*unit_income
    total_cost = cost_pozo *count
    revenues = income_total - total_cost
    return revenues




In [58]:
## Calculo de ganancia para los 200 pozos mas rentables de cada región
ingreso_unitario = 4500
pozos_count = 200
costo_por_pozo = 100000000/200

print("Ingresos por región de los pozos más rentables 'Modelo de regresion lineal':")
     
print("Región 0:", round(revenues(pd.Series(predictions_valid_0), pd.Series(target_valid_0), pozos_count, ingreso_unitario, costo_por_pozo)))

print("Región 1:", round(revenues(pd.Series(predictions_valid_1), pd.Series(target_valid_1), pozos_count, ingreso_unitario, costo_por_pozo)))

print("Región 2:", round(revenues(pd.Series(predictions_valid_2), pd.Series(target_valid_2), pozos_count, ingreso_unitario, costo_por_pozo)))


Ingresos por región de los pozos más rentables 'Modelo de regresion lineal':
Región 0: 33208260
Región 1: 24150867
Región 2: 27103500


 - Si se toman los 200 pozos mas rentable , la región con mayores ingresos en la region 0, seguida por la región 3 
 
 

## Calculo de riesgos y ganancias para cada región <a class="anchor" id="chapter5"></a>

In [59]:
#Calculo de ganancias e intervalos de confianza mediante bootstrapping

#region 0

values_0=[]
state = np.random.RandomState(12345)
for i in range(1000):
    subsample = pd.Series(predictions_valid_0).sample(500, replace = True, random_state = state)#generacion de muestra
    values_0.append(revenues(subsample,pd.Series(target_valid_0), 200, 4500, costo_por_pozo)) #calculo de ingreso para 200 pozos mas rentables de la muestra

    
#intervalo de confianza
values_0 = pd.Series(values_0)   
lower_0 = values_0.quantile(0.025) 
upper_0 = values_0.quantile(0.975)
mean_0= values_0.mean()
loss_prob_0 = (values_0 < 0).mean() * 100 # porcentaje de predicciones negativas

print("La media de ingresos para los mejores pozos de la region 0 es de", "{:,}".format(round(mean_0)))
print("Con un intervalo de confianza del 95%", "{:,}".format(round(lower_0)), ' - ',"{:,}".format(round(upper_0)))
print("Probabilidad de perdida%:", loss_prob_0)




La media de ingresos para los mejores pozos de la region 0 es de 3,961,650
Con un intervalo de confianza del 95% -1,112,155  -  9,097,669
Probabilidad de perdida%: 6.9


In [60]:
#Calculo de ganancias e intervalos de confianza mediante bootstrapping

#region 1

values_1=[]
state = np.random.RandomState(12345)
for i in range(1000):
    subsample = pd.Series(predictions_valid_1).sample(500, replace = True, random_state = state)#generacion de muestra
    values_1.append(revenues(subsample,pd.Series(target_valid_1), 200, 4500, costo_por_pozo)) #calculo de ingreso para 200 pozos mas rentables de la muestra

    
#intervalo de confianza
values_1 = pd.Series(values_1)   
lower_1 = values_1.quantile(0.025) 
upper_1 = values_1.quantile(0.975)
mean_1 = values_1.mean()
loss_prob_1 = (values_1 < 0).mean() * 100 #promedio de predicciones negativas

print("La media de ingresos para los mejores pozos de la region 0 es de", "{:,}".format(round(mean_1)))
print("Con un intervalo de confianza del 95%", "{:,}".format(round(lower_1)), ' - ',"{:,}".format(round(upper_1)))
print("Probabilidad de perdida%:", loss_prob_1)



La media de ingresos para los mejores pozos de la region 0 es de 4,560,451
Con un intervalo de confianza del 95% 338,205  -  8,522,895
Probabilidad de perdida%: 1.5


In [61]:
#region 2

values_2=[]
state = np.random.RandomState(12345)
for i in range(1000):
    subsample = pd.Series(predictions_valid_2).sample(500, replace = True, random_state = state)#generacion de muestra
    values_2.append(revenues(subsample, pd.Series(target_valid_2), 200, 4500, costo_por_pozo)) #calculo de ingreso para 200 pozos mas rentables de la muestra

    
#intervalo de confianza
values_2 = pd.Series(values_2)   
lower_2 = values_2.quantile(0.025) 
upper_2 = values_2.quantile(0.975)
mean_2 = values_2.mean()
loss_prob_2 = (values_2 < 0).mean() * 100 #promedio de predicciones negativas

print("La media de ingresos para los mejores pozos de la region 0 es de", "{:,}".format(round(mean_2)))
print("Con un intervalo de confianza del 95%", "{:,}".format(round(lower_2)), ' - ',"{:,}".format(round(upper_2)))
print("Probabilidad de perdida%:", loss_prob_2)




La media de ingresos para los mejores pozos de la region 0 es de 4,044,039
Con un intervalo de confianza del 95% -1,633,504  -  9,503,596
Probabilidad de perdida%: 7.6


- La region 1 es la que cuenta con la menor probabilidad de perdida y el mayor beneficio promedio

## Conclusiones <a class="anchor" id="chapter6"></a>

- En un primer paso: se cargaron los datos, se separaron los datos de cada region en conjunto de entrenamiento y validación, y se escalaron las características numericas para entrenar al modelo

- Se entrenó un modelo de regresión lineal para cada región y se calculó el RECM, resultando el menor valor de RECN el correspndiente a la region 1. LOs valores de RECM para las regiones 0 y 2, resultaron muy altos (casi la mitad de la media)
 
- La media de reservas por pozo predecida por el modelo, en los 3 casos, resulto inferior al valor minimo de reservas requerida para superar la inversión por pozo  

- Sin embargo, al considerar los 200 pozos más rentables estimados por los modelos, la ganancia total estimada resulto ser positiva en las 3 regiones

- La region con la mayor ganancia estimada para los 200 pozos mas rentables resulto ser la region 0, con un valor cercano a los 33 millones de usd, seguida por la región 2 con 27 millones de uds  y luego la 1 24 Millones de usd estimados de ingreso total 
 
- Utilizando la tecnica de bootstrapping, el mayor beneficio medio obtenido para los 200 pozos más rentables, es el que corresponde a la región 1 con un valor en torno a 4,5 millones de dolares y un con un intervalo de confianza positivo, es decir que arroja ganancia con una certeza del 95%. El riesgo de perdida calculado es del 1,5%
 
- Se concluye que la region 1 es la más indicada para el desarrollo de pozoz petroleros